In [ ]:
!pip install xgboost tensorflow

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the dataset
try:
    df = pd.read_csv('safety_grid_data.csv')
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print("Error: 'safety_grid_data.csv' was not found.")
    print("Please run the data generation script from the previous step first.")

# --- 2. Define Features (X) and Target (y) ---
# The target is what we want to predict: 'safety_category'
# We subtract 1 because models need labels to start at 0 (0, 1, 2)
y = df['safety_category'] - 1

# The features are all columns except the target and lat/lon
X = df.drop(['safety_category', 'latitude', 'longitude'], axis=1)

# --- 3. Split the Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

Dataset loaded successfully!
Training set shape: (400, 7)
Test set shape: (100, 7)


In [ ]:
# Define which columns are which type
numerical_features = ['poi_count', 'dist_to_police', 'dist_to_transport',
                      'incident_count_30d', 'avg_incident_severity']

categorical_features = ['area_type', 'is_holiday']

# Create the preprocessor
# StandardScaler scales numerical data (like 'dist_to_police')
# OneHotEncoder converts categorical data (like 'area_type') into numbers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Keep any other columns
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# 1. Define the models
models = {
    "Logistic Regression (Multinomial)": LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# 2. Create and train a pipeline for each model
# We'll store the results here
model_pipelines = {}
model_results = {}

print("\n--- Training Scikit-Learn Models ---")

for name, model in models.items():
    # Create the full pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    # Train the model
    print(f"Training {name}...")
    pipeline.fit(X_train, y_train)

    # Save the trained pipeline
    model_pipelines[name] = pipeline

    # 4. Evaluate the model
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    model_results[name] = accuracy

    print(f"--- Results for {name} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, y_pred, target_names=['1 (Safe)', '2 (Moderate)', '3 (Unsafe)'], zero_division=0))
    print("-"*40 + "\n")

print("\n--- Model Accuracy Comparison ---")
for name, acc in model_results.items():
    print(f"{name}: {acc:.4f}")


--- Training Scikit-Learn Models ---
Training Logistic Regression (Multinomial)...
--- Results for Logistic Regression (Multinomial) ---
Accuracy: 0.9800
              precision    recall  f1-score   support

    1 (Safe)       1.00      1.00      1.00        37
2 (Moderate)       0.97      0.97      0.97        31
  3 (Unsafe)       0.97      0.97      0.97        32

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100

----------------------------------------

Training Random Forest...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


--- Results for Random Forest ---
Accuracy: 0.9800
              precision    recall  f1-score   support

    1 (Safe)       1.00      1.00      1.00        37
2 (Moderate)       0.97      0.97      0.97        31
  3 (Unsafe)       0.97      0.97      0.97        32

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100

----------------------------------------

Training Gradient Boosting...
--- Results for Gradient Boosting ---
Accuracy: 0.9800
              precision    recall  f1-score   support

    1 (Safe)       1.00      0.97      0.99        37
2 (Moderate)       0.94      1.00      0.97        31
  3 (Unsafe)       1.00      0.97      0.98        32

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100

----------------------------------------


--- Model Accurac

In [ ]:
import xgboost as xgb

print("\n--- Training XGBoost Model ---")

# Create the XGBoost pipeline
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', xgb.XGBClassifier(
                                  objective='multi:softmax',
                                  num_class=3,
                                  use_label_encoder=False,
                                  eval_metric='mlogloss',
                                  random_state=42
                              ))])

# Train the model
xgb_pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred_xgb = xgb_pipeline.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
model_results['XGBoost'] = accuracy_xgb

print(f"--- Results for XGBoost ---")
print(f"Accuracy: {accuracy_xgb:.4f}")
print(classification_report(y_test, y_pred_xgb, target_names=['1 (Safe)', '2 (Moderate)', '3 (Unsafe)']))
print("-" * 40 + "\n")


--- Training XGBoost Model ---
--- Results for XGBoost ---
Accuracy: 0.9700
              precision    recall  f1-score   support

    1 (Safe)       1.00      0.97      0.99        37
2 (Moderate)       0.91      1.00      0.95        31
  3 (Unsafe)       1.00      0.94      0.97        32

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100

----------------------------------------



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [17:49:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

print("\n--- Training Artificial Neural Network (ANN) ---")

# 1. Preprocess the data for the ANN
# We need to fit the preprocessor on the training data
# and then transform both training and test data.
# We also convert to a dense array (.toarray()) as ANNs don't like sparse matrices.
X_train_ann = preprocessor.fit_transform(X_train)
X_test_ann = preprocessor.transform(X_test)

# 2. One-hot encode the target variable (y)
# (e.g., '1' becomes [0, 1, 0])
y_train_ann = to_categorical(y_train, num_classes=3)
y_test_ann = to_categorical(y_test, num_classes=3)

# 3. Build the ANN Model Architecture
input_shape = X_train_ann.shape[1] # Number of input features

ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    Dropout(0.3), # Dropout layer to prevent overfitting
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax') # 3 output neurons (one for each class), softmax for probabilities
])

ann_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', # Use this loss for one-hot encoded labels
    metrics=['accuracy']
)

print(ann_model.summary())

# 4. Train the ANN
history = ann_model.fit(
    X_train_ann,
    y_train_ann,
    epochs=50, # Train for 50 cycles
    batch_size=32,
    validation_split=0.1, # Use 10% of training data for validation
    verbose=0 # Hides the long training log
)

print("ANN training complete.")

# 5. Evaluate the ANN
loss, accuracy_ann = ann_model.evaluate(X_test_ann, y_test_ann)
model_results['ANN'] = accuracy_ann

# Convert softmax predictions back to class labels (e.g., [0.1, 0.8, 0.1] -> 1)
y_pred_ann_probs = ann_model.predict(X_test_ann)
y_pred_ann = np.argmax(y_pred_ann_probs, axis=1)

print(f"--- Results for ANN ---")
print(f"Test Accuracy: {accuracy_ann:.4f}")
print(classification_report(y_test, y_pred_ann, target_names=['1 (Safe)', '2 (Moderate)', '3 (Unsafe)']))
print("-" * 40 + "\n")


--- Training Artificial Neural Network (ANN) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,947 (11.51 KB)

 Trainable params: 2,947 (11.51 KB)

 Non-trainable params: 0 (0.00 B)

None
ANN training complete.
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9847 - loss: 0.0863 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
--- Results for ANN ---
Test Accuracy: 0.9800
              precision    recall  f1-score   support

    1 (Safe)       1.00      1.00      1.00        37
2 (Moderate)       0.97      0.97      0.97        31
  3 (Unsafe)       0.97      0.97      0.97        32

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100

----------------------------------------



In [ ]:
print("\n--- 🏆 Final Model Performance Summary 🏆 ---")

# Sort the results by accuracy
sorted_results = sorted(model_results.items(), key=lambda item: item[1], reverse=True)

for name, acc in sorted_results:
    print(f"{name}: {acc:.4f}")

# You can now see which model is the winner!


--- 🏆 Final Model Performance Summary 🏆 ---
ANN: 0.9800
Logistic Regression (Multinomial): 0.9800
Random Forest: 0.9800
Gradient Boosting: 0.9800
XGBoost: 0.9700


In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

print("\n--- 🌳 Starting Hyperparameter Tuning for Random Forest... ---")

# 1. Create the pipeline
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', RandomForestClassifier(random_state=42))])

# 2. Define the parameter search space
# These are the "knobs" we'll turn
param_dist_rf = {
    'classifier__n_estimators': [100, 200, 300, 500],
    'classifier__max_depth': [10, 20, 30, None],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# 3. Set up the Randomized Search
# n_iter=25: Tries 25 different random combinations
# cv=3: Uses 3-fold cross-validation
# n_jobs=-1: Uses all available CPU cores
rf_random_search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions=param_dist_rf,
    n_iter=25,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# 4. Run the search
rf_random_search.fit(X_train, y_train)

# 5. Show the best parameters
print("\n--- Best Parameters for Random Forest ---")
print(rf_random_search.best_params_)

# 6. Evaluate the new best model
best_rf_model = rf_random_search.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"\nTuned Random Forest Accuracy: {accuracy_rf:.4f}\n")
print(classification_report(y_test, y_pred_rf, target_names=['1 (Safe)', '2 (Moderate)', '3 (Unsafe)']))


--- 🌳 Starting Hyperparameter Tuning for Random Forest... ---
Fitting 3 folds for each of 25 candidates, totalling 75 fits

--- Best Parameters for Random Forest ---
{'classifier__n_estimators': 200, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 1, 'classifier__max_depth': 30}

Tuned Random Forest Accuracy: 0.9800

              precision    recall  f1-score   support

    1 (Safe)       1.00      0.97      0.99        37
2 (Moderate)       0.94      1.00      0.97        31
  3 (Unsafe)       1.00      0.97      0.98        32

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100



In [ ]:
print("\n--- 📈 Starting Hyperparameter Tuning for XGBoost... ---")

# 1. Create the pipeline
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', xgb.XGBClassifier(
                                  objective='multi:softmax',
                                  num_class=3,
                                  use_label_encoder=False,
                                  eval_metric='mlogloss',
                                  random_state=42
                              ))])

# 2. Define the parameter search space
param_dist_xgb = {
    'classifier__n_estimators': [100, 200, 300, 500],
    'classifier__max_depth': [3, 5, 7, 10],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'classifier__colsample_bytree': [0.7, 0.8, 0.9, 1.0],
    'classifier__subsample': [0.7, 0.8, 0.9, 1.0]
}

# 3. Set up the Randomized Search
xgb_random_search = RandomizedSearchCV(
    xgb_pipeline,
    param_distributions=param_dist_xgb,
    n_iter=25,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# 4. Run the search
xgb_random_search.fit(X_train, y_train)

# 5. Show the best parameters
print("\n--- Best Parameters for XGBoost ---")
print(xgb_random_search.best_params_)

# 6. Evaluate the new best model
best_xgb_model = xgb_random_search.best_estimator_
y_pred_xgb = best_xgb_model.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

print(f"\nTuned XGBoost Accuracy: {accuracy_xgb:.4f}\n")
print(classification_report(y_test, y_pred_xgb, target_names=['1 (Safe)', '2 (Moderate)', '3 (Unsafe)']))


--- 📈 Starting Hyperparameter Tuning for XGBoost... ---
Fitting 3 folds for each of 25 candidates, totalling 75 fits

--- Best Parameters for XGBoost ---
{'classifier__subsample': 0.7, 'classifier__n_estimators': 500, 'classifier__max_depth': 10, 'classifier__learning_rate': 0.1, 'classifier__colsample_bytree': 0.8}

Tuned XGBoost Accuracy: 0.9900

              precision    recall  f1-score   support

    1 (Safe)       1.00      1.00      1.00        37
2 (Moderate)       0.97      1.00      0.98        31
  3 (Unsafe)       1.00      0.97      0.98        32

    accuracy                           0.99       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100



/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [17:58:05] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# --- 1. Re-split Data to Get Test Set Coordinates ---
# We need to do the split again on the *full* DataFrame
# so we can get the latitude and longitude for our test points.

# This assumes 'df' is your full 500-point DataFrame
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Create the X_test features (without lat/lon/target)
# This is what the model needs for prediction
X_test_for_prediction = df_test.drop(['safety_category', 'latitude', 'longitude'], axis=1)


# --- 2. Get Predictions from Your Best Model ---
# IMPORTANT: Use the variable name of your best-tuned model from the previous step.
# I'll assume it's 'best_xgb_model'. If you named it 'best_rf_model', use that.
# We'll use the variable 'random_search' from the tuning step to get the best model.

# Use this line if you ran the XGBoost tuning:
best_model = xgb_random_search.best_estimator_

# Use this line if you ran the Random Forest tuning and it was better:
# best_model = rf_random_search.best_estimator_

print("Making predictions on the test data...")
y_pred = best_model.predict(X_test_for_prediction)


# --- 3. Prepare the Data for Plotting ---
# Create a new DataFrame from our test set
df_map_data = df_test.copy()

# Add the model's predictions (0,1,2) as a new column
# We add 1 to make them (1,2,3) for clarity
df_map_data['predicted_safety_category'] = y_pred + 1

# Convert the numerical category to a meaningful label
def get_label(category):
    if category == 1:
        return "Predicted: Safe"
    elif category == 2:
        return "Predicted: Use Caution"
    else:
        return "Predicted: Unsafe"

df_map_data['Safety Level'] = df_map_data['predicted_safety_category'].apply(get_label)

print("\n--- Data Ready for Map Visualization ---")
print(df_map_data[['latitude', 'longitude', 'Safety Level']].head())


# --- 4. Create the Interactive Map ---
print("\nGenerating interactive map...")
fig = px.scatter_mapbox(
    df_map_data,
    lat="latitude",
    lon="longitude",
    color="Safety Level",  # Color points based on the model's prediction
    hover_name="Safety Level",
    hover_data={
        "latitude": ":.4f",
        "longitude": ":.4f",
        "area_type": True,
        "incident_count_30d": True,
        "dist_to_police": True
    },
    size_max=15,
    zoom=9.5, # Zoomed in on Delhi
    center={"lat": 28.6139, "lon": 77.2090}, # Center of Delhi
    title="ML Model Safety Predictions for Delhi (Test Data)",
    color_discrete_map={  # Manually set the colors
        "Predicted: Safe": "green",
        "Predicted: Use Caution": "orange",
        "Predicted: Unsafe": "red"
    }
)

# Use an open-source map style (no API key needed)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0, "t":50, "l":0, "b":10})

# --- 5. Show the Map ---
fig.show()

Making predictions on the test data...

--- Data Ready for Map Visualization ---
      latitude  longitude            Safety Level
361  28.721644  77.026644  Predicted: Use Caution
73   28.807731  77.111043  Predicted: Use Caution
374  28.435594  76.812693  Predicted: Use Caution
155  28.521028  76.863790       Predicted: Unsafe
104  28.853783  76.860318       Predicted: Unsafe

Generating interactive map...


In [ ]:
!pip install openrouteservice folium

In [ ]:
import pandas as pd
import folium
import openrouteservice
from openrouteservice import convert

# --- 1. Setup ---
# Use your key from your code. (Please generate a new one after this!)
api_key = 'eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjhjOWZjN2ZjZGE2MTRhYTNhYTIyYjZkNjk4YTEwNzJjIiwiaCI6Im11cm11cjY0In0='
client = openrouteservice.Client(key=api_key)

# --- 2. Define Our Points ---
# Using Delhi coordinates (Note: ORS uses Lon, Lat)
start_point = (77.2167, 28.6139) # India Gate (Lon, Lat)
end_point = (77.1025, 28.7041)   # Rohini (Lon, Lat)

# *** THIS IS WHERE YOUR ML MODEL COMES IN ***
# Your model predicted that a certain area is unsafe. We'll use its coordinate.
# Let's pretend your model found an "Unsafe" (Category 3) spot in the middle.
unsafe_area_center = (77.1691, 28.6692) # Karol Bagh area (Lon, Lat)

# --- 3. Create an "Avoid" Zone ---
# We create a GeoJSON polygon (a small box) around the unsafe spot.
box_size = 0.01
unsafe_polygon = {
    "type": "Polygon",
    "coordinates": [[
        [unsafe_area_center[0] - box_size, unsafe_area_center[1] - box_size],
        [unsafe_area_center[0] - box_size, unsafe_area_center[1] + box_size],
        [unsafe_area_center[0] + box_size, unsafe_area_center[1] + box_size],
        [unsafe_area_center[0] + box_size, unsafe_area_center[1] - box_size],
        [unsafe_area_center[0] - box_size, unsafe_area_center[1] - box_size]
    ]]
}
avoid_options = {'avoid_polygons': unsafe_polygon}

# --- 4. Get Both Routes ---
print("Requesting routes from ORS...")
coords = [start_point, end_point]

# Call 1: The Fastest Route
route_fastest = client.directions(coords, profile='driving-car')
decoded_fastest = convert.decode_polyline(route_fastest['routes'][0]['geometry'])

# Call 2: The "Safest" Route (avoids our unsafe polygon)
route_safest = client.directions(coords, profile='driving-car', options=avoid_options)
decoded_safest = convert.decode_polyline(route_safest['routes'][0]['geometry'])

print("Routes received!")

# --- 5. Create the Map ---
# Folium uses (Lat, Lon)
map_center = [28.65, 77.15] # Center of Delhi
m = folium.Map(location=map_center, zoom_start=11)

# Plot the "Fastest Route" (Blue)
folium.PolyLine(
    locations=[(pt[1], pt[0]) for pt in decoded_fastest['coordinates']],
    color='blue',
    weight=5,
    opacity=0.8,
    popup="Fastest Route",
    name="Fastest Route"
).add_to(m)

# Plot the "Safest Route" (Green)
folium.PolyLine(
    locations=[(pt[1], pt[0]) for pt in decoded_safest['coordinates']],
    color='green',
    weight=5,
    opacity=0.8,
    popup="Safest Route",
    name="Safest Route"
).add_to(m)

# --- 6. Add Markers ---
# Start and End points (Folium needs Lat, Lon)
folium.Marker(
    [start_point[1], start_point[0]],
    popup="START",
    icon=folium.Icon(color="green", icon="play")
).add_to(m)

folium.Marker(
    [end_point[1], end_point[0]],
    popup="END",
    icon=folium.Icon(color="red", icon="stop")
).add_to(m)

# Mark the Unsafe Area
folium.GeoJson(
    data=unsafe_polygon,
    style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'fillOpacity': 0.3},
    name="Unsafe Zone (from ML Model)"
).add_to(m)

# Add a layer control to toggle routes
folium.LayerControl().add_to(m)

print("Map created. Displaying m...")
m # This will display the map in your Colab notebook

Requesting routes from ORS...
Routes received!
Map created. Displaying m...
